# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:
* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [1]:
# Your code here
import pandas as pd
import sqlite3
conn = sqlite3.connect('data.sqlite')

## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [11]:
# Your code here
pd.read_sql("""
SELECT firstname, lastname
FROM employees AS e
JOIN offices AS o
     ON e.officeCode = o.officeCode
WHERE o.city = 'Boston';
""", conn)

,firstName,lastName
0,Julie,Firrelli
1,Steve,Patterson


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [13]:
# Your code here
pd.read_sql("""
SELECT o.officeCode,
       o.city,
       COUNT(e.employeeNumber) AS num_employees
FROM offices AS o
LEFT JOIN employees AS e
     USING (officeCode)
GROUP BY officeCode
HAVING num_employees = 0;
""", conn)

,officeCode,city,num_employees
0,27,Boston,0


## Write 3 questions of your own and answer them

In [14]:
# Answers will vary

# Example question: 
"""
How many customers are there per office?
"""

'\nHow many customers are there per office?\n'

In [17]:
"""
Question 1
"""

# Your code here
"""
How many customers are there per office?
"""
pd.read_sql("""
SELECT o.officeCode,
       o.city,
       COUNT(c.customerNumber) AS num_customers
FROM offices AS o
JOIN employees AS e
     USING (officeCode)
JOIN customers AS c
     ON e.employeeNumber = c.salesRepEmployeeNumber
GROUP BY officeCode;
""", conn)

,officeCode,city,num_customers
0,1,San Francisco,12
1,2,Boston,12
2,3,NYC,15
3,4,Paris,29
4,5,Tokyo,5
5,6,Sydney,10
6,7,London,17


In [21]:
"""
Question 2: 
"""
#How many orders every office received?

# Your code here
pd.read_sql("""
SELECT o.officeCode,
       o.city,
       COUNT(od.orderNumber) AS num_orders
FROM offices AS o
JOIN employees AS e
     USING (officeCode)
JOIN customers AS c
     ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders AS od
     USING (customerNumber)
GROUP BY officeCode;
""", conn)

,officeCode,city,num_orders
0,1,San Francisco,48
1,2,Boston,32
2,3,NYC,39
3,4,Paris,106
4,5,Tokyo,16
5,6,Sydney,38
6,7,London,47


In [29]:
"""
Question 3
"""
# How many orders of each product were placed?

# Your code here
pd.read_sql("""
SELECT productCode AS product, COUNT(od.orderNumber) AS num_orders
FROM orderdetails AS od
JOIN products AS p
     USING (productCode)
GROUP BY productCode
ORDER BY num_orders;
""", conn)

,product,num_orders
0,S18_4933,24
1,S24_2887,24
2,S18_1589,25
3,S18_1749,25
4,S18_2248,25
...,...,...
104,S700_2834,28
105,S700_3167,28
106,S700_4002,28
107,S72_1253,28


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [30]:
# Your code here
pd.read_sql("""
SELECT e.firstName,
       e.lastName,
       p.productName
FROM employees AS e
JOIN customers AS c
     ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders AS o
     USING (customerNumber)
JOIN orderdetails
     USING (orderNumber)
JOIN products AS p
     USING (productCode)
GROUP BY productCode;
""", conn)

,firstName,lastName,productName
0,Leslie,Jennings,1969 Harley Davidson Ultimate Chopper
1,Leslie,Jennings,1952 Alpine Renault 1300
2,Leslie,Jennings,1996 Moto Guzzi 1100i
3,Leslie,Jennings,2003 Harley-Davidson Eagle Drag Bike
4,Leslie,Jennings,1972 Alfa Romeo GTA
...,...,...,...
104,Leslie,Jennings,The Titanic
105,Leslie,Jennings,The Queen Mary
106,Leslie,Jennings,American Airlines: MD-11S
107,Leslie,Jennings,Boeing X-32A JSF


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [32]:
# Your code here
pd.read_sql("""
SELECT e.lastName,
       e.firstName,
       SUM(orderDetails.quantityOrdered) AS num_product
FROM employees AS e
JOIN customers AS c
     ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders AS o
     USING (customerNumber)
JOIN orderdetails
     USING (orderNumber)
GROUP BY lastName, firstName
ORDER BY lastName;
""", conn)

,lastName,firstName,num_product
0,Bondur,Loui,6186
1,Bott,Larry,8205
2,Castillo,Pamela,9290
3,Firrelli,Julie,4227
4,Fixter,Andy,6246
5,Gerard,Martin,4180
6,Hernandez,Gerard,14231
7,Jennings,Leslie,11854
8,Jones,Barry,7486
9,Marsh,Peter,6632


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [35]:
# Your code here
pd.read_sql("""
SELECT e.lastName,
       e.firstName,
       COUNT(orderDetails.productCode) AS num_product
FROM employees AS e
JOIN customers AS c
     ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders
     USING (customerNumber)
JOIN orderdetails
     USING (orderNumber)
GROUP BY lastName, firstName
HAVING num_product > 200
ORDER BY lastName;
""", conn)

,lastName,firstName,num_product
0,Bott,Larry,236
1,Castillo,Pamela,272
2,Hernandez,Gerard,396
3,Jennings,Leslie,331
4,Jones,Barry,220
5,Vanauf,George,211


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!